## BHS Data Consolidation

**Author:** CFOLKERS <br>
**Ministry, Division, Branch:** WLRS, GeoBC, Geospatial Services <br>
**Created Date:** 2023 12 14 <br>
**Updated Date:**  2024 01 10 <br>
**Description:** A tool to consolidate BHS data for the KB region and update the original dataset <br>

In [1]:
import sqlalchemy 
import cx_Oracle
from getpass import getpass
from osgeo import ogr
from osgeo import gdal
import pandas
import arcpy
import os
import openpyxl as pyxl
from datetime import date

#get oracle username and password 
username= input('Enter BCGW user name: ')
password =getpass(prompt='Enter BCGW password: ')

#gdbs
exsitng_gdb=r'BHS_Data.gdb'
temp_gdb=r'T:\BHS_Temp.gdb'

#XLSL tracking sheet
tracker=r'BHS_Consolidated_Dataset_Tracking.xlsx'

In [2]:
#create oracle connection and engine
dialect=''
sql_driver=''
hostname=''
port= 1
service_name=''

#  Connection string
oracle_connection_string_fmt = (
    'oracle+cx_oracle://{username}:{password}@' +
    cx_Oracle.makedsn('{hostname}', '{port}', service_name='{service_name}')
)
url = oracle_connection_string_fmt.format(
    username=username, password=password, 
    hostname=hostname, port=port, 
    service_name=service_name,
)

engine: sqlalchemy.engine.Engine = sqlalchemy.create_engine(url, echo=True)
conn = engine.connect()
metadata=sqlalchemy.MetaData()

2024-01-10 09:01:42,469 INFO sqlalchemy.engine.Engine select sys_context( 'userenv', 'current_schema' ) from dual
2024-01-10 09:01:42,470 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-01-10 09:01:42,547 INFO sqlalchemy.engine.Engine select value from nls_session_parameters where parameter = 'NLS_NUMERIC_CHARACTERS'
2024-01-10 09:01:42,549 INFO sqlalchemy.engine.Engine [raw sql] {}


In [4]:
#query observation data set in bounding box (Kootenay Boundary)
sql_obvs="""
select 
  b.SURVEY_OBSERVATION_ID as Observation_ID,
  b.Species_Code,
  b.ANIMAL_ID,
  b.SEX,
  b.OBSERVED_NUMBER,
  b.OBSERVATION_DATETIME as Date_Time,
  b.OBSERVATION_YEAR as Year,
  b.OBSERVATION_MONTH as Month,
  b.OBSERVATION_DAY as Day,
  b.INVENTORY_METHOD,
  b.UTM_SOURCE,
  b.UTM_ZONE,
  b.UTM_EASTING,
  b.UTM_Northing,
  b.PROJECT_NAME,
  b.PROJECT_ID,
  b.SURVEY_NAME,
  b.SURVEY_ID,
  SDO_UTIL.TO_WKTGEOMETRY(b.GEOMETRY)AS WKT_GEOMETRY
from
  WHSE_WILDLIFE_INVENTORY.SPI_SURVEY_OBS_ALL_SP b
  
where b.species_code = 'M-OVCA'

and 

SDO_ANYINTERACT (GEOMETRY,
	SDO_GEOMETRY(2003, 3005, NULL,
		SDO_ELEM_INFO_ARRAY(1,1003,3),
		SDO_ORDINATE_ARRAY(1493647.6,466354.8,1889017.9,862609.6) 
	)
) = 'TRUE'
  
"""
#query telemetry data set in bounding box (kootenay boundary)
sql_tele="""
select 
  b.SURVEY_OBSERVATION_ID as Observation_ID,
  b.Species_Code,
  b.ANIMAL_ID,
  b.SEX,
  b.OBSERVED_NUMBER,
  b.OBSERVATION_DATETIME as Date_Time,
  b.OBSERVATION_YEAR as Year,
  b.OBSERVATION_MONTH as Month,
  b.OBSERVATION_DAY as Day,
  b.UTM_ZONE,
  b.UTM_EASTING,
  b.UTM_Northing,
  b.PROJECT_NAME,
  b.PROJECT_ID,
  b.SURVEY_NAME,
  b.SURVEY_ID,
  SDO_UTIL.TO_WKTGEOMETRY(b.GEOMETRY)AS WKT_GEOMETRY
from
  WHSE_WILDLIFE_INVENTORY.SPI_TELEMETRY_OBS_ALL_SP b
  
where b.species_code = 'M-OVCA'

and 

SDO_ANYINTERACT (GEOMETRY,
	SDO_GEOMETRY(2003, 3005, NULL,
		SDO_ELEM_INFO_ARRAY(1,1003,3),
		SDO_ORDINATE_ARRAY(1493647.6,466354.8,1889017.9,862609.6) 
	)
) = 'TRUE'
  
"""
# b.INVENTORY_METHOD
  # b.UTM_SOURCE,

In [5]:
#execute queries
obvs_df=pandas.read_sql_query(sql_obvs, conn)
tele_df=pandas.read_sql_query(sql_tele, conn)
#Merge results into one table 
frames=[obvs_df,tele_df]
result_df=pandas.concat(frames)

#split data frame into two, pre and post 1998
post_1998,pre_1998=[x for _, x in result_df.groupby(result_df['year']<=1998)]

#split pre and post 1998 data frames into pre and post, winter and movment based on dates
post_winter= post_1998.query('month <=3 or month =12' and 'day >=15')
print(len(post_winter.index))

post_movement=post_1998.query('month = 11 or month =12' and 'day <15 or month in (4,5)')
print(len(post_movement.index))

pre_winter= pre_1998.query('month <=3 or month =12' and 'day >=15')
print(len(pre_winter.index))

pre_movement= pre_1998.query('month = 11 or month =12' and 'day <15 or month in (4,5)')
print(len(pre_movement.index))

# OG function to define date ranges in data here for reference m=month, d=day
"""def cat (m, d):
    if m <=3:
        return 'Winter'
    elif m == 12 and d >=15:
        return 'Winter'
    elif m == 11:
        return 'Movement'
    elif m == 12 and d <15:
        return 'Movement'
    elif m ==4 or m ==5:
        return 'Movement'"""

2024-01-10 09:02:03,777 INFO sqlalchemy.engine.Engine 
select 
  b.SURVEY_OBSERVATION_ID as Observation_ID,
  b.Species_Code,
  b.ANIMAL_ID,
  b.SEX,
  b.OBSERVED_NUMBER,
  b.OBSERVATION_DATETIME as Date_Time,
  b.OBSERVATION_YEAR as Year,
  b.OBSERVATION_MONTH as Month,
  b.OBSERVATION_DAY as Day,
  b.INVENTORY_METHOD,
  b.UTM_SOURCE,
  b.UTM_ZONE,
  b.UTM_EASTING,
  b.UTM_Northing,
  b.PROJECT_NAME,
  b.PROJECT_ID,
  b.SURVEY_NAME,
  b.SURVEY_ID,
  SDO_UTIL.TO_WKTGEOMETRY(b.GEOMETRY)AS WKT_GEOMETRY
from
  WHSE_WILDLIFE_INVENTORY.SPI_SURVEY_OBS_ALL_SP b
  
where b.species_code = 'M-OVCA'

and 

SDO_ANYINTERACT (GEOMETRY,
	SDO_GEOMETRY(2003, 3005, NULL,
		SDO_ELEM_INFO_ARRAY(1,1003,3),
		SDO_ORDINATE_ARRAY(1493647.6,466354.8,1889017.9,862609.6) 
	)
) = 'TRUE'
  

2024-01-10 09:02:03,778 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-01-10 09:03:09,797 INFO sqlalchemy.engine.Engine 
select 
  b.SURVEY_OBSERVATION_ID as Observation_ID,
  b.Species_Code,
  b.ANIMAL_ID,
  b.SEX,
  b.OBSER

"def cat (m, d):\n    if m <=3:\n        return 'Winter'\n    elif m == 12 and d >=15:\n        return 'Winter'\n    elif m == 11:\n        return 'Movement'\n    elif m == 12 and d <15:\n        return 'Movement'\n    elif m ==4 or m ==5:\n        return 'Movement'"

In [6]:
#create temp gdb if it does not exist
fgdb=os.path.join("T:","BHS_Temp.gdb")
if not arcpy.Exists(fgdb):
    arcpy.management.CreateFileGDB("T:","BHS_Temp.gdb")
else: 
    print('gdb exists')

arcpy.env.workspace=fgdb

#names for temp lyrs 
tmp_lyrs=['Post_1998_Winter_temp','Pre_1998_Winter_temp','Post_1998_Movement_temp','Pre_1998_Movement_temp']
dfs=[post_winter,pre_winter, post_movement, pre_movement]
#create dictonary  of col name:type
column_types = post_winter.dtypes.apply(lambda x: x.name).to_dict()

#create temp fc if they do not exist and add cols to it 
for t,d in zip(tmp_lyrs,dfs):
    if not arcpy.Exists(t):
        arcpy.management.CreateFeatureclass(fgdb,
            t,
            'POINT',
            spatial_reference=arcpy.SpatialReference(3005))
        print(f'{t} created')
    else: 
        print(f'{t} already exists')
    #add cols from df to fc
    for col in column_types:
        if column_types[col] == 'int64' or column_types[col] == 'float64':
            arcpy.management.AddField(t, col, 'DOUBLE')
        elif column_types[col] == 'object':
            arcpy.management.AddField(t, col, 'TEXT')
        elif column_types[col] == 'datetime64[ns]':
            arcpy.management.AddField(t, col, 'DATE')
    #add points from df to fc        
    with arcpy.da.InsertCursor(t, ['SHAPE@'] + list(d.columns)) as cursor:
        for index, row in d.iterrows():
            # Create a point geometry from WKT
            point = arcpy.FromWKT(row['wkt_geometry'], arcpy.SpatialReference(3005))  # Update with the appropriate coordinate system code
            row_nm=row.tolist()
            values = [point] + row_nm
            cursor.insertRow(values)


Post_1998_Winter_temp created
Pre_1998_Winter_temp created
Post_1998_Movement_temp created
Pre_1998_Movement_temp created


In [31]:
conn.close()

In [4]:

#FCs
e_post_winter=os.path.join(exsitng_gdb, 'Post_1998_Winter')
e_post_movement=os.path.join(exsitng_gdb, 'Post_1998_Movement')
e_pre_winter=os.path.join(exsitng_gdb, 'Pre_1998_Winter')
e_pre_movement =os.path.join(exsitng_gdb, 'Pre_1998_Movement')

t_post_winter=os.path.join(temp_gdb, 'Post_1998_Winter_temp')
t_post_movement=os.path.join(temp_gdb, 'Post_1998_Movement_temp')
t_pre_winter=os.path.join(temp_gdb, 'Pre_1998_Winter_temp')
t_pre_movement =os.path.join(temp_gdb, 'Pre_1998_Movement_temp')

fc_dict={e_post_winter:t_post_winter, e_post_movement:t_post_movement, e_pre_winter:t_pre_winter, e_pre_movement:t_pre_movement}



In [6]:
# Create or read xlsx
if not os.path.exists(tracker):
    wb= pyxl.Workbook()
    ws=wb.active
    ws1= wb.create_sheet('Data_Tracking',0)
    header=['Date updated', 'feature class', 'Number of new features', 'Source', 'Updated by']
    header_style=pyxl.styles.Font(size=12, bold=True)
    ws1=wb['Data_Tracking']
    ws1.append(header)
    for cell in ws1["1:1"]:
        cell.font=header_style
    wb.save(tracker)
else:
    wb= pyxl.load_workbook(tracker)
    ws1=wb['Data_Tracking']


In [7]:
for key in fc_dict:
    sel_feats=arcpy.management.SelectLayerByLocation(fc_dict[key], 'WITHIN', key)
    print('select by layer')
    del_feats=int(arcpy.management.GetCount(sel_feats).getOutput (0))
    print(f'{del_feats} features overlap, removing')
    arcpy.management.DeleteFeatures(fc_dict[key])
    
    feats=int(arcpy.management.GetCount(fc_dict[key]).getOutput (0))
    print(f'{feats} remaining, adding to master...')
    if feats>0:
        field_mappings = arcpy.FieldMappings()
        field_mappings.addTable(fc_dict[key])
        field_mappings.addTable(key)
        print('field mapping?')

        arcpy.management.Append(fc_dict[key], key, schema_type="NO_TEST", field_mapping=field_mappings)
        print('append master class')
        fc_name=key.split(r'\\')
        fc_name=fc_name[-1]
        # how do we get the source? flag from  
        xlsx_append=[date.today, fc_name, feats, 'Comming soon', username]
        print(xlsx_append)
        ws1.append(xlsx_append)
    elif feats == 0:
        print( 'no new feats')

    break
    


select by layer
0 features overlap, removing
0 remaining, adding to master...
no new feats
